# CO2-Ausstoss von Personenwagen
In diesem Notebook werden Informationen des Bundesamts für Strassen aufbereitet. Der Datensatz ist sehr umfangreich. Damit trotzdem effizient damit gearbeitet werden kann, werden bloss die für unsere Fragestellungen relevanten Daten eingelesen beziehungsweise behalten. Das genaue Vorgehen kann anhand des untenstehenden Python-Codes nachvollzogen werden. Die Rohdaten sind allerdings nicht verfügbar, da diese (kostenpflichtig) beim Bundesamt für Strassen bezogen werden müssen. 

In [1]:
# Importieren der nötigen Bibliotheken. 
import pandas as pd
import numpy as np
import os
from tqdm.auto import tqdm
tqdm.pandas()

/home/this/.local/lib/python3.7/site-packages/tqdm/std.py:658: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [2]:
# Vor dem Import: Welche Informationen sind im Originaldatensatz vorhanden? 
!cat BEST_R-20200201.txt | head -n 1


cat: Schreibfehler: Datenübergabe unterbrochen (broken pipe)


In [3]:
# Die für unsere Fragestellung relevanten Spalten werden ausgewählt.
columns = ['Fahrzeugart', 'Leistung', 'Treibstoff', 'Erstinverkehrsetzung_Jahr', 'Altersklasse_Halter', 
           'Ort', 'BFS-Gemeinde-Nr', 'Halterart_Code', 'CO2', 'Staat', 'Inverkehrsetzung_Kanton']

In [16]:
# Einlesen der Daten. 
df = pd.read_csv("BEST_R-20200201.txt", sep='\t', usecols=columns, engine='c')

In [5]:
# Uns interessieren einzig die Personenwagen. Alle anderen Daten werden ausgefiltert.
df = df[df['Fahrzeugart'] == 'Personenwagen'].copy()

In [6]:
# Im Datensatz sind Autos enthalten, die für den Export bestimmt sind oder nicht auf die Strassen sollen. 
# Diese sind für uns irrelevant. Wir behalten einzig Fahrzeuge mit einem Schweizer Nummernschild.
df = df[df['Staat'] == 'Schweiz'].copy()

In [7]:
# Bei wie vielen Personenwagen fehlt eine Angabe zum CO2-Wert? 
len(df[df['CO2'].isna()])

381565

In [8]:
# Bei wie vielen Prozent aller Autos fehlt eine Angabe zum CO2-Wert?
100 * len(df[df['CO2'].isna()]) / len(df)

8.329633117607482

In [9]:
# Autos, bei denen weder beim Treibstoff noch beim CO2 ein Wert erfasst sind, werden entfernt. 
df = df.dropna(subset=['Treibstoff', 'CO2'], how='all')

In [10]:
# Verbrennungsmotoren, bei denen kein CO2-Wert vorhanden ist, werden entfernt. 
df['Treibstoff'] = df['Treibstoff'].str.strip()
treibstoff = ['Benzin', 'Diesel', 'Methanol', 'Flüssiggas (LPG) / Benzin',
           'Benzin / Elektrisch', 'Diesel / Elektrisch', 'Benzin / Alkohol (Ethanol)', 'Erdgas (CNG) / Benzin', 
           'Gas (CNG/GPL)', 'Flüssiggas (LPG)', 'Erdgas (CNG)', 
           'Alkohol (Ethanol)', 'Petrol']
df = df[~((df['Treibstoff'].isin(treibstoff)) & (df['CO2'].isna()))]

In [11]:
# Bei den übrigen Autos -- vorwiegend mit Elektroantrieb -- wird der fehlende CO2-Wert auf 0 gesetzt. 
# Damit werden diese Fahrzeuge in der Auswertung miteinbezogen. 
for i, r in df[df['CO2'].isna()].iterrows():
    df.at[i, 'CO2'] = 0

In [12]:
# In den Daten gibts diverse Einträge zu emmissionslosen Verbrennungsmotoren. Wie viele sind es? 
len(df[((df['CO2'] == 0) & ((df['Treibstoff'] == 'Benzin') | (df['Treibstoff'] == 'Diesel')))])

2835

In [13]:
# Verbrennungsmotoren, die kein CO2 verursachen, gibts leider nicht. Die entsprechenden Einträge werden entfernt. 
df = df[~((df['CO2'] == 0) & (df['Treibstoff'] == 'Benzin'))]
df = df[~((df['CO2'] == 0) & (df['Treibstoff'] == 'Diesel'))]

In [14]:
# Im Datensatz sind Gemeinden enthalten, bei denen die nicht existierende Gemeindenummer 0 eingetragen wurde. 
# Wir versuchen anhand der übrigen Daten die richtigen Nummern einzutragen. 
df['BFS-Gemeinde-Nr'] = df['BFS-Gemeinde-Nr'].apply(lambda x: int(str(x).replace('\'', '')))

for i, r in tqdm(df[df['BFS-Gemeinde-Nr'] == 0].iterrows()):
    try:
        b = df[df['Ort'] == r['Ort']]['BFS-Gemeinde-Nr'].value_counts().tolist()[0]
        df.at[i, 'BFS-Gemeinde-Nr'] = int(b)
    except: 
        pass

# Weitere Korrektur: Bei allen Einträgen zu Ponte Cremenaga, einem Weiler in der Gemeinde Monteggio, 
# fehlen die BfS-Nummern. 
for i, r in tqdm(df[df['Ort'].str.lower() == 'ponte cremenaga'].iterrows()):
    df.at[i, 'BFS-Gemeinde-Nr'] = 5202

In [15]:
# Welchen Gemeinden sind mehrere BfS-Nummern zugeordnet? 
df_test = pd.DataFrame(df.groupby(by=['Ort', 'Inverkehrsetzung_Kanton'])['BFS-Gemeinde-Nr'].nunique())
df_test = df_test[df_test['BFS-Gemeinde-Nr'] != 1]
df_test = df_test.reset_index()

# Eine Spalte für die korrigierte BFS-Nummer wird eingefügt. 
for i, r in tqdm(df_test.iterrows()):
    bfs = df[(df['Ort'] == r['Ort']) & (df['Inverkehrsetzung_Kanton'] == r['Inverkehrsetzung_Kanton'])]['BFS-Gemeinde-Nr'].value_counts().index.tolist()[0]
    df_test.at[i, 'bfs_korr'] = bfs
    
df_test.sort_values(by='BFS-Gemeinde-Nr', ascending=False)

,Ort,Inverkehrsetzung_Kanton,BFS-Gemeinde-Nr,bfs_korr
269,Lausanne,VD,27,5586.0
163,Embrach-Embraport,ZH,23,23.0
275,Le Bémont,JU,15,6741.0
206,Gland,VD,14,5721.0
582,Yverdon-les-Bains,VD,14,5938.0
...,...,...,...,...
262,La Plaine,GE,2,6620.0
261,La Neuveville,BE,2,723.0
260,La Forclaz VD,VD,2,5410.0
259,La Croix-de-Rozon,GE,2,6605.0


Im Datensatz existieren Gemeinden, denen mehr als eine BFS-Nummer zugeordnet wurden. Dabei scheint es sich um Fehler zu handeln. Wir können nicht beurteilen, ob das Feld mit dem Gemeindenamen oder jenes mit der BFS-Nummer korrekt ist. Deshalb verzichten wir auf einen Korrekturversuch. Als Grundlage für die Auswertungen im nächsten Notebook dienen also weitgehend die amtlich erfassten Werte.

In [59]:
# Exportieren der gefilterten Komplettdaten. 
df.to_csv('daten/co2_daten_gefiltert.csv')

In [60]:
# Exportieren der gefilterten Daten gruppiert nach Gemeinde. 
df_export = df.groupby(by='BFS-Gemeinde-Nr').agg({'Ort': 'first', 'CO2': 'mean', 'Treibstoff': 'count'})
df_export.columns = ['ort', 'CO2', 'anz_fahrzeuge']
df_export.to_csv('daten/co2_daten_gefiltert_gruppiert.csv')